# Loading Libraries

In [ ]:
import os
os.listdir('../input/lognormalizefeaturesetstogether')
print(os.listdir('../input/lognormalizefeaturesetstogetherwextrafeats'))

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
from catboost import Pool, CatBoostClassifier
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler

# Extracting Features from train set

In [ ]:
traindf=pd.read_csv('../input/lognormalizefeaturesetstogetherwextrafeats/igTrain.csv')
#del traindf['distmod']

print(traindf.shape)
traindf.head()


## Get y information

In [ ]:

full_train = traindf
del traindf
full_train.shape

if 'target' in full_train.columns:
    y = full_train['target']
    ydf=pd.DataFrame()
    ydf['target']=full_train['target']
    if 'object_id' in full_train.columns:
        ydf['object_id']=full_train['object_id']
    
    del full_train['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

print('Unique classes : ', classes)

In [ ]:
ydf.head()

# Standard Scaling the input (imp.)

In [ ]:
#import copy
#full_train_new = copy.deepcopy(full_train)
#ss = StandardScaler()
#full_train_ss = ss.fit_transform(full_train_new)

if 'object_id' in full_train:
    oof_df = full_train[['object_id']]
    del full_train['object_id']

full_train_ss=full_train.values

In [ ]:

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd

#modify to work with kfold
#def smoteAdataset(Xig, yig, test_size=0.2, random_state=0):
def smoteAdataset(Xig_train, yig_train, Xig_test, yig_test):
    
        
    sm=SMOTE(random_state=2)
    Xig_train_res, yig_train_res = sm.fit_sample(Xig_train, yig_train.ravel())

        
    return Xig_train_res, pd.Series(yig_train_res), Xig_test, pd.Series(yig_test)

In [ ]:

    
#train_mean = full_train.mean(axis=0)
#full_train.fillna(train_mean, inplace=True)

folds = StratifiedKFold(n_splits=12, shuffle=True, random_state=1)

# Deep Learning Begins...

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.utils import to_categorical
import tensorflow as tf
from keras import backend as K
import keras
from keras import regularizers
from collections import Counter
from sklearn.metrics import confusion_matrix

In [ ]:
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/69795
def mywloss(y_true,y_pred):  
    yc=tf.clip_by_value(y_pred,1e-15,1-1e-15)
    loss=-(tf.reduce_mean(tf.reduce_mean(y_true*tf.log(yc),axis=0)/wtable))
    return loss

In [ ]:
def multi_weighted_logloss(y_ohe, y_p):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    classes = [6, 16, 53, 65, 92]
    class_weight = {6: 1, 16: 1, 53: 1, 65: 1, 92: 1}
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1-1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set 
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos    
    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss

# Defining simple model in keras

In [ ]:
K.clear_session()
def build_model(dropout_rate=0.35,activation='relu'):
    start_neurons = 1024
    # create model
    model = Sequential()
    model.add(Dense(start_neurons, input_dim=full_train_ss.shape[1], activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(start_neurons//2,activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(start_neurons//4,activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(start_neurons//8,activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate/2))
    
    model.add(Dense(start_neurons//16,activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate/2))
    
    model.add(Dense(len(classes), activation='softmax'))
    return model

In [ ]:
unique_y = np.unique(y)
class_map = dict()
for i,val in enumerate(unique_y):
    class_map[val] = i
        
y_map = np.zeros((y.shape[0],))
y_map = np.array([class_map[val] for val in y])
y_categorical = to_categorical(y_map)

# Calculating the class weights

In [ ]:
y_count = Counter(y_map)
wtable = np.zeros((len(unique_y),))
for i in range(len(unique_y)):
    wtable[i] = y_count[i]/y_map.shape[0]

In [ ]:
def plot_loss_acc(history):
    plt.plot(history.history['loss'][1:])
    plt.plot(history.history['val_loss'][1:])
    plt.title('model loss')
    plt.ylabel('val_loss')
    plt.xlabel('epoch')
    plt.legend(['train','Validation'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['acc'][1:])
    plt.plot(history.history['val_acc'][1:])
    plt.title('model Accuracy')
    plt.ylabel('val_acc')
    plt.xlabel('epoch')
    plt.legend(['train','Validation'], loc='upper left')
    plt.show()

In [ ]:
clfs = []
oof_preds = np.zeros((len(full_train_ss), len(classes)))
epochs = 600
batch_size = 100
for fold_, (trn_, val_) in enumerate(folds.split(y_map, y_map)):
    checkPoint = ModelCheckpoint("./keras.model",monitor='val_loss',mode = 'min', save_best_only=True, verbose=0)
    x_train, y_train = full_train_ss[trn_], y_categorical[trn_]
    x_valid, y_valid = full_train_ss[val_], y_categorical[val_]
    
    #trn_xa, y_train, val_xa, y_valid=smoteAdataset(x_train, pd.Series(y_train), x_valid, pd.Series(y_valid))
    #x_train=pd.DataFrame(data=trn_xa, columns=x_train.columns)

    #x_valid=pd.DataFrame(data=val_xa, columns=x_valid.columns)

    model = build_model(dropout_rate=0.5,activation='tanh')    
    model.compile(loss=mywloss, optimizer='adam', metrics=['accuracy'])
    history = model.fit(x_train, y_train,
                    validation_data=[x_valid, y_valid], 
                    epochs=epochs,
                    batch_size=batch_size,shuffle=True,verbose=0,callbacks=[checkPoint])       
    
    plot_loss_acc(history)
    
    print('Loading Best Model')
    model.load_weights('./keras.model')
    # # Get predicted probabilities for each class
    oof_preds[val_, :] = model.predict_proba(x_valid,batch_size=batch_size)
    print(multi_weighted_logloss(y_valid, model.predict_proba(x_valid,batch_size=batch_size)))
    clfs.append(model)
    
score=multi_weighted_logloss(y_categorical,oof_preds)

print('MULTI WEIGHTED LOG LOSS : %.5f ' % score)

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# Compute confusion matrix
#cnf_matrix = confusion_matrix(y_map, np.argmax(oof_preds,axis=-1))
#np.set_printoptions(precision=2)

In [ ]:
import pandas as pd
#sample_sub = pd.read_csv('../input/PLAsTiCC-2018/sample_submission.csv')
#class_names = list(sample_sub.columns[1:-1])
#del sample_sub;
class_names=['class_6', 'class_16','class_53',
            'class_65','class_92']

gc.collect()

In [ ]:
# Plot non-normalized confusion matrix
#plt.figure(figsize=(12,12))
#foo = plot_confusion_matrix(cnf_matrix, classes=class_names,normalize=True,
#                      title='Confusion matrix')


# Test Set Predictions

In [ ]:
testdf=pd.read_csv('../input/lognormalizefeaturesetstogetherwextrafeats/igTest.csv')
#del testdf['distmod']

if 'object_id' in testdf:
    objidTest=pd.DataFrame()
    objidTest.loc[:,'object_id']=testdf.loc[:,'object_id']
    del testdf['object_id']

testdf.shape

testdf.describe()



In [ ]:
#df[df.columns] = scaler.fit_transform(df[df.columns])
#ss = StandardScaler()
#testdf[testdf.columns]=ss.fit_transform(testdf[testdf.columns])
#testdf.describe()

testdf_ss=testdf.values


In [ ]:
testdf['object_id']=objidTest['object_id']
del testdf['object_id']
testdf.head()

In [ ]:
def predict_test(testdf, objidTest, clfs, features, class_names):
    
    preds = None
    for clf in clfs:
        if preds is None:
            preds = clf.predict_proba(testdf[features]) / len(clfs)
        else:
            preds += clf.predict_proba(testdf[features]) / len(clfs)
            
    preds_99 = np.ones(preds.shape[0])
    
    
    for i in range(preds.shape[1]):
        preds_99 *= (1 - preds[:, i])

    # Create DataFrame from predictions
    preds_df = pd.DataFrame(preds, columns=class_names)
    preds_df['object_id'] = objidTest['object_id']
    preds_df['class_99'] = 0.14 * preds_99 / np.mean(preds_99)
    
    return preds_df

    

In [ ]:
pdf=predict_test(testdf, objidTest, clfs, full_train.columns, class_names)


In [ ]:
pdf.head()

In [ ]:
#from Scirpus discussion:

def GenUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)



In [ ]:
from datetime import datetime as dt
filename = 'subm_{:.6f}_{}.csv'.format(score, 
                 dt.now().strftime('%Y-%m-%d-%H-%M'))
print('save to {}'.format(filename))
# TEST


In [ ]:
# get a list of columns
cols = list(pdf)
# move the column to head of list using index, pop and insert
#cols.insert(0, cols.pop(cols.index('object_id')))
#pdf = pdf.loc[:, cols]



feats = ['class_6', 'class_16',  'class_53', 'class_65', 'class_92']

y = pd.DataFrame()
y['mymean'] = pdf[feats].mean(axis=1)
y['mymedian'] = pdf[feats].median(axis=1)
y['mymax'] = pdf[feats].max(axis=1)

pdf['class_99'] = GenUnknown(y)
pdf.head()

In [ ]:
pdf.describe()

In [ ]:
pdf.to_csv(filename, index=False)